<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 6
<br>
<b>Deadline:</b> March 24, 2021 (Wednesday) 23:00
</div>

# Exercise 4.2 Recommender system

In this exercise, your task is to design a recommender system.

## Learning goals:
* Practise tuning a neural network model by using different regularization methods.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tools
import data

In [2]:
skip_training = True  # Set this flag to True before validation and submission
skip_tests = True

In [3]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Ratings dataset

We will train the recommender system on the dataset in which element consists of three values:
* `user_id` - id of the user (the smallest user id is 1)
* `item_id` - id of the movie (the smallest item id is 1)
* `rating` - rating given by the user to the item (ratings are integer numbers between 1 and 5.

The recommender system need to predict the rating for any given pair of `user_id` and `item_id`.

We measure the quality of the predicted ratings using the mean-squared error (MSE) loss:
$$
  \frac{1}{N}\sum_{i=1}^N (r_i - \hat{r}_i)^2
$$
where $r_i$ is a real rating and $\hat{r}_i$ is a predicted one.

Note: The predicted rating $\hat{r}_i$ does not have to be an integer number.

In [7]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

In [8]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)

In [9]:
# Print one sample from the dataset
x = trainset[0]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=1, item_id=1, rating=5


# Model

You need to design a recommender system model with the API described in the cell below.

Hints on the model architecture:
* You need to use [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding) layer to convert inputs `user_ids` and `item_ids` into reasonable representations. The idea of the embedding layer is that we want to represent similar users with values that are close to each other. The original representation as integers is not good for that. By using the embedding layer, we can learn such useful representations automatically.

### Model tuning

In this exercise, you need to tune the architecture of your model to achieve the best performance on the provided test set. You will notice that overfitting is a severe problem for this data: The model can easily overfit the training set producing poor accuracy on the out-of-training (test) data.

You need to find an optimal combination of the hyperparameters, with some hyperparameters corresponding to the regularization techniques that we studied in the lecture.

The hyperparameters that you are advised to consider:
* Learning rate value and learning rate schedule (decresing the learning rate often has positive effect on the model performance)
* Number of training epochs
* Network size
* Weight decay
* Early stopping
* Dropout
* Increase amount of data:
  * Data augmentation
  * Injecting noise

You can tune the hyperparameters by, for example, grid search, random search or manual tuning.

Note:
* The number of points that you will get from this exercise depends on the MSE loss on the test set:
  * below 1.00: 1 point
  * below 0.95: 2 points
  * below 0.92: 3 points
  * below 0.90: 4 points
  * below 0.89: 5 points
  * below 0.88: 6 points 

In [10]:
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items,
                 embed_size = 100,
                 drop_embed = 0.02, drop_hl = 0.2,
                 n_layers = 2, layer_size = 100):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """
        # YOUR CODE HERE
        super(RecommenderSystem, self).__init__()
        
        self.users = nn.Embedding(n_users+1, embed_size)
        self.items = nn.Embedding(n_items+1, embed_size)
        
        if drop_hl is not None:
            self.drop = nn.Dropout(drop_embed)
        else:
            self.drop = nn.Sequential()
        
        def hiden_layers(n_in):
            for _ in range(n_layers):
                yield nn.Linear(n_in, layer_size)
                yield nn.ReLU()
                if drop_hl is not None:
                    yield nn.Dropout(drop_hl)
                n_in = layer_size
        
        self.hl = nn.Sequential(*list(hiden_layers(embed_size * 2)))
        
        self.fc = nn.Linear(layer_size,1)
        #raise NotImplementedError()
        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        # YOUR CODE HERE
        usr = self.users(user_ids)
        itm = self.items(item_ids)
        x = torch.cat([usr, itm], dim=1)
        x = self.drop(x)
        x = self.hl(x)
        y = torch.sigmoid(self.fc(x)).flatten()*5
        return y
        #raise NotImplementedError()

You can test the shapes of the model outputs using the function below.

In [11]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


In [12]:
# This cell is reserved for testing

## Train the model

You need to train a recommender system using **only the training data.** Please use the test set to select the best model: the model that generalizes best to out-of-training data.

In [13]:
# Create the model
model = RecommenderSystem(trainset.n_users, trainset.n_items)

In [14]:
class EarlyStopping:
    def __init__(self, tolerance, patience):
        """
        Args:
          patience (int):    Maximum number of epochs with unsuccessful updates.
          tolerance (float): We assume that the update is unsuccessful if the validation error is larger
                              than the best validation error so far plus this tolerance.
        """
        self.tolerance = tolerance
        self.patience = patience
    
    def stop_criterion(self, val_errors):
        """
        Args:
          val_errors (iterable): Validation errors after every update during training.
        
        Returns: True if training should be stopped: when the validation error is larger than the best
                  validation error obtained so far (with given tolearance) for patience epochs (number of consecutive epochs for which the criterion is satisfied).
                 
                 Otherwise, False.
        """
        if len(val_errors) <= self.patience:
            return False

        min_val_error = min(val_errors)
        val_errors = np.array(val_errors[-self.patience:])
        return val_errors[-1] > min_val_error + self.tolerance

In [15]:
# This is the function to compute the loss:
def compute_loss(model, x1, x2, y):
    model.eval()
    with torch.no_grad():
        outputs = model.forward(x1,x2)
        loss = F.mse_loss(outputs, y)
        return loss.cpu().numpy()

# This is the function to print the progress during training
def print_progress(epoch, train_error, val_error):
    print('Epoch {}: Train error: {:.4f}, Test error: {:.4f}'.format(
        epoch, train_error, val_error))

In [16]:
# Implement the training loop in this cell
def train_model(embed_size = 100,
                drop_embed = 0.02, drop_hl = 0.2,
                n_layers = 2, layer_size = 100,
                epochs = 50, lr = 0.0001, wd = 0.001,
                early_stop = True, tolerance = 0.001, patience = 15,
                plot = False):    
    
    model = RecommenderSystem(trainset.n_users, trainset.n_items, embed_size, 
                              drop_embed, drop_hl,
                              n_layers, layer_size)
    
    if not skip_training:
        # YOUR CODE HERE
        model.zero_grad()
        lossFunc = F.mse_loss
        
        if wd is not None:
            optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = wd)
        else: 
            optimizer = torch.optim.Adam(model.parameters(), lr = lr)

        if early_stop:
            eStop = EarlyStopping(tolerance=tolerance, patience=patience)
        
        train_errors = []
        val_errors = []
        
        for epoch in range(epochs):
            model.train()
            for user_ids, item_ids, ratings in trainloader:
                optimizer.zero_grad()
                ratings = ratings.to(torch.float32)
                y = model.forward(user_ids, item_ids)
                loss = lossFunc(y, ratings)
                loss.backward()
                optimizer.step()

            train_error = compute_loss(model, trainset[:][0], trainset[:][1], trainset[:][2])
            train_errors.append(train_error)
            
            val_error = compute_loss(model, testset[:][0], testset[:][1], testset[:][2])
            val_errors.append(val_error)
            
            print_progress(epoch, train_error, val_error)
            
            if early_stop and eStop.stop_criterion(val_errors):
                print('Stop after %d epochs' % epoch)
                break
        
        if plot:
            if early_stop:
                N = epoch+1
            else:
                N = epochs

            plt.plot(range(N), train_errors, label = 'training loss')
            plt.plot(range(N), val_errors, label = 'validation loss')
            
            print('Final Test error:',val_errors[-1])
        
        return model
    
model = train_model()

In [17]:
if not skip_tests:
    #Without regularization
    train_model(embed_size = 100,
                drop_embed = None, drop_hl = None,
                n_layers = 2, layer_size = 100,
                epochs = 10, lr = 0.0001, wd = None,
                early_stop = False,
                plot = True)

In [18]:
if not skip_tests:
    #With weight decay
    train_model(embed_size = 100,
                drop_embed = None, drop_hl = None,
                n_layers = 2, layer_size = 100,
                epochs = 10, lr = 0.0001, wd = 0.001,
                early_stop = False,
                plot = True)

In [19]:
if not skip_tests:
    #With weight decay and dropping after embedding
    train_model(embed_size = 100,
                drop_embed = 0.2, drop_hl = None,
                n_layers = 2, layer_size = 100,
                epochs = 10, lr = 0.0001, wd = 0.001,
                early_stop = False,
                plot = True)

In [20]:
if not skip_tests:
    #With weight decay, dropping after embedding and dropping between layers
    train_model(embed_size = 100,
                drop_embed = 0.2, drop_hl = 0.2,
                n_layers = 2, layer_size = 100,
                epochs = 10, lr = 0.0001, wd = 0.001,
                early_stop = False,
                plot = True)

In [21]:
if not skip_tests:
    #With weight decay, dropping after embedding and dropping between layers with more (50) epochs
    train_model(embed_size = 100,
                drop_embed = 0.2, drop_hl = 0.2,
                n_layers = 2, layer_size = 100,
                epochs = 50, lr = 0.0001, wd = 0.001,
                early_stop = False,
                plot = True)

In [22]:
if not skip_tests:
    #With weight decay, dropping after embedding, dropping between layers and early stop
    train_model(embed_size = 100,
                drop_embed = 0.02, drop_hl = 0.2,
                n_layers = 2, layer_size = 100,
                epochs = 50, lr = 0.0001, wd = 0.001,
                early_stop = True, tolerance = 0.001, patience = 15,
                plot = True)

In [23]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(model, 'recsys.pth', confirm=True)

In [24]:
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    tools.load_model(model, 'recsys.pth', device)

Model loaded from recsys.pth.


In [25]:
# Compute the accuracy on the test set
test_error = compute_loss(model, testset[:][0], testset[:][1], testset[:][2])
print('Test error of the network on the test set: %.3f' % test_error)

assert test_error <= 1, "Bad model. Test error ({:.3f})".format(test_error)
print('Success')

if test_error >= 0.95:
    points = 1
elif test_error >= 0.92:
    points = 2
elif test_error >= 0.90:
    points = 3
elif test_error >= 0.89:
    points = 4
elif test_error >= 0.88:
    points = 5
else:
    points = 6
    
print('Points:', points, '/ 6')

Test error of the network on the test set: 0.857
Success
Points: 6 / 6


In [26]:
# This cell is reserved for grading

In [27]:
# This cell is reserved for grading

In [28]:
# This cell is reserved for grading

In [29]:
# This cell is reserved for grading

In [30]:
# This cell is reserved for grading

In [31]:
# This cell is reserved for grading

In [32]:
# This cell is reserved for grading